In [ ]:
# Import modules
import pyodbc
import pandas as pd
import time
from datetime import datetime

In [ ]:
# Connect to SQL Server
conn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server};'
                     'Server=server_name.com;'
                     'Database=database_name;'
                     'Trusted_Connection=no;'
                     'ApplicationIntent=READONLY;'
                     'UID=uid_name;'
                     'PWD=abcdefghijk')

In [ ]:
start = time.time()

sqlString =     """
                SELECT
                    tblInventory.code AS id,
                    tblValues_1.current_value AS system,
                    tblValues_2.current_value AS district,
                    tblValues_3.current_value AS county,
                    tblValues_4.current_value AS main_span_below_ground,
                    tblValues_5.current_value AS maj_span_below_ground,
                    tblValues_6.current_value AS min_span_below_ground,
                    tblValues_7.current_value AS condition_rating_1,
                    tblValues_8.current_value AS condition_rating_2,
                    tblValues_9.current_value AS condition_rating_3,
                    tblValues_10.current_value AS condition_rating_4,
                    tblValues_11.current_value AS railing,
                    tblValues_12.current_value AS condition_status,
                    tblValues_13.current_value AS operational_status,
                    tblValues_14.current_value AS latitude,
                    tblValues_15.current_value AS longitude
                    
                FROM (((((((((((((((tblInventory
                /*System*/
                LEFT JOIN tblValues AS tblValues_1
                    ON (tblInventory.id = tblValues_1.id
                        AND (tblValues_1.field_id = 100)))
                /*District*/
                LEFT JOIN tblValues AS tblValues_2
                    ON (tblInventory.id = tblValues_2.id
                        AND (tblValues_2.field_id = 200)))
                /*County*/
                LEFT JOIN tblValues AS tblValues_3
                    ON (tblInventory.id = tblValues_3.id
                        AND (tblValues_3.field_id = 300)))
                /*Main Span Below Ground*/
                LEFT JOIN tblValues AS tblValues_4
                    ON (tblInventory.id = tblValues_4.id
                        AND (tblValues_4.field_id = 400)))
                /*Major Span Below Ground*/
                LEFT JOIN tblValues AS tblValues_5
                    ON (tblInventory.id = tblValues_5.id
                        AND (tblValues_5.field_id = 500)))
                /*Minor Span Below Ground*/
                LEFT JOIN tblValues AS tblValues_6
                    ON (tblInventory.id = tblValues_6.id
                        AND (tblValues_6.field_id = 600)))
                /*Condition Rating 1*/
                LEFT JOIN tblValues AS tblValues_7
                    ON (tblInventory.id = tblValues_7.id
                        AND (tblValues_7.field_id = 700)))
                /*Condition Rating 2*/
                LEFT JOIN tblValues AS tblValues_8
                    ON (tblInventory.id = tblValues_8.id
                        AND (tblValues_8.field_id = 800)))
                /*Condition Rating 3*/
                LEFT JOIN tblValues AS tblValues_9
                    ON (tblInventory.id = tblValues_9.id
                        AND (tblValues_9.field_id = 900)))
                /*Condition Rating 4*/
                LEFT JOIN tblValues AS tblValues_10
                    ON (tblInventory.id = tblValues_10.id
                        AND (tblValues_10.field_id = 1000)))
                /*Rail*/
                LEFT JOIN tblValues AS tblValues_11
                    ON (tblInventory.id = tblValues_11.id
                        AND (tblValues_11.field_id = 1100)))
                /*Condition Status*/
                LEFT JOIN tblValues AS tblValues_12
                    ON (tblInventory.id = tblValues_12.id
                        AND (tblValues_12.field_id = 1200)))
                /*Operational Status*/
                LEFT JOIN tblValues AS tblValues_13
                    ON (tblInventory.id = tblValues_13.id
                        AND (tblValues_13.field_id = 1300)))
                /*Latitude*/
                LEFT JOIN tblValues AS tblValues_14
                    ON (tblInventory.id = tblValues_14.id
                        AND (tblValues_14.field_id = 1400)))
                /*Longitude*/
                LEFT JOIN tblValues AS tblValues_15
                    ON (tblInventory.id = tblValues_15.id
                        AND (tblValues_15.field_id = 1500)))
                WHERE (tblInventory.ab_id = 1)
                    AND (LEFT(tblInventory.code, 1) IN('0','1','2'))
                    AND (tblInventory.code <> '0')
                    AND LEN(tblInventory.code) IN(14, 15)
                    AND ((tblValues_4.current_value = '3')
                        OR (tblValues_5.current_value = '3')
                        OR (tblValues_6.current_value = '3'))
                    OR ((tblValues_7.current_value IN('0','1','2','3','4'))
                        OR (tblValues_8.current_value IN('0','1','2','3','4'))
                        OR (tblValues_9.current_value IN('0','1','2','3','4'))
                        OR (tblValues_10.current_value IN('0','1','2','3','4')))
                    OR (tblValues_11.current_value = '0')
                    OR ((tblValues_12.current_value IN('N','0'))
                        AND (tblValues_13.current_value IN('P','B')))
                ORDER BY id ASC
                """

# Read SQL into dataframe
df = pd.read_sql(sqlString, conn)

end = time.time()
print('Time elapsed = ' + str(end - start))

# Verify the query columns and shape
print(df.head(5))
print('Number of records = ' + str(df.shape[0]))
print('Number of columns = ' + str(df.shape[1]))

In [ ]:
# Function for finding timber codes in the data
def timber_piles(row):
    if row['main_span_below_ground'] == '3':
        return 'timber'
    elif row['maj_span_below_ground'] == '3':
        return 'timber'
    elif row['min_span_below_ground'] == '3':
        return 'timber'
    else:
        return None
    
df['timber'] = df.apply(timber_piles, axis=1)

In [ ]:
# Function for finding poor conditions in the data
def poor_ratings(row):
    if row['condition_rating_1'] < '5':
        return 'poor'
    elif row['condition_rating_2'] < '5':
        return 'poor'
    elif row['condition_rating_3'] < '5':
        return 'poor'
    elif row['condition_rating_4'] < '5':
        return 'poor'
    else:
        return 'good'
    
df['poor_rating'] = df.apply(poor_bridges, axis=1)

In [ ]:
# Function for finding deficient rails in the data
def deficient_rail(row):
    if row['railing'] == '0':
        return 'deficient'
    else:
        return 'good'
    
df['railing_status'] = df.apply(deficient_rail, axis=1)

In [ ]:
# Function for finding SSLO in the data
def sslo_records(row):
    if row['condition_status'] == '0':
        if row['operational_status'] == 'P':
            return 'SSLO'
        elif row['operational_status'] == 'B':
            return 'SSLO'
    elif row['condition_status'] == 'N':
        if row['operational_status'] == 'P':
            return 'SSLO'
        elif row['operational_status'] == 'B':
            return 'SSLO'
    else:
        return None
    
df['sslo_status'] = df.apply(sslo_records, axis=1)

In [ ]:
# Verify functions successfully created new fields
print(df.head(20))
print('Number of records = ' + str(df.shape[0]))
print('Number of columns = ' + str(df.shape[1]))

In [ ]:
# Export dataframe to excel
now = datetime.today()
datestamp = datetime(now.year, now.month, now.day)strftime('%m-%d-%Y')

writer = pd.ExcelWriter(r'C:\data_exports\export_' + datestamp + '.xlsx', engine='openpyxl')

def paste_to_sheet(df, new_sheet_name):
    df.to_excel(writer,
        sheet_name=new_sheet_name,
        index=True
        freeze_panes=(1,0),
        verbose=True)
    
# Convert dataframe to xlsxWriter Excel object
paste_to_sheet(df, 'Data')

# Close Pandas Excel writer and output the Excel file
writer.save()
conn.close()
print('Complete!')